In [ ]:
import os
import numpy as np
sscurves = []
model_in = []
modified = []
random_par = []
for filename in os.listdir(r'./ssresults'):
    curve = np.load(r'./ssresults/' + filename)
    sscurves.append(curve[1])
    f = open(r'./polymer/' + filename[0:-8] + '.txt')
    parameters = []
    for line in f:
        parameters.append(float(line))
    modified.append(parameters)
# [length,{A},{B},num_A,num_B,max_A,max_B]
for i in modified:
    random_par.append([i[0],int(i[0]*i[1]),int(i[0]*(1-i[1]))])
for i in range(len(random_par)):
    if(modified[i][2]==0):
        random_par[i].append(1)
    else:
        random_par[i].append(2)
    random_par[i].append(1)
    random_par[i].append(int((1-modified[i][2])*random_par[i][1]))
    random_par[i].append(random_par[i][2])
block = np.array(random_par).astype('float32')
y = np.array(sscurves).astype('float32')

In [ ]:
def countnum(pattern,monomer):
    return pattern.count(monomer)

def num_block(pattern,monomer):
    cnt = 0
    for i in range(1,len(pattern)):
        if(pattern[i]!=monomer and pattern[i-1]==monomer):
            cnt+=1
    if(pattern[i]==monomer):
            cnt+=1
    return cnt

def max_block(pattern,monomer):
    max_block = -1
    count = 0
    for i in range(len(pattern)):
        if(pattern[i]==monomer):
            count+=1
        else:
            if(count>max_block):
                max_block = count
            count = 0
    if(count>max_block):
        max_block = count
    return max_block

In [ ]:
import random
# Design Space
# [length,{A},{B},num_A,num_B,max_A,max_B]
sample = 1505
random.seed(1126)
random_copolymer = [""]*sample
design_space = [[]]*sample
num_beads = 60
fA = 0.1
for i in range(sample):
    design_space[i]=[num_beads,int(num_beads*fA),num_beads-int(num_beads*fA)]
    fA+=0.015
    if(i!=0 and (i+1)%50==0):
        num_beads+=2
        fA=0.1
for i in range(sample):
    polymer = ""
    for j in range(design_space[i][1]):
        polymer += "A"
    for j in range(design_space[i][2]):
        polymer += "B"
    polymer = list(polymer)
    random.shuffle(polymer)
    random_copolymer[i] = "".join(polymer)
for i in range(sample):
    design_space[i].append(num_block(random_copolymer[i],"A"))
    design_space[i].append(num_block(random_copolymer[i],"B"))
    design_space[i].append(max_block(random_copolymer[i],"A"))
    design_space[i].append(max_block(random_copolymer[i],"B"))

In [ ]:
# load query data
query = []
for i in range(1,41):
    query.append(np.load(f"./query_data/iter_{i}.npy",allow_pickle=True)[0])

In [ ]:
# histogram for query data and the whole design space
import matplotlib.pyplot as plt
name = ["number of monomers","number of A monomers","number of B monomers","mean cluster size of A monomers","mean cluster size of B monomers","max cluster size of A monomers","max cluster size of B monomers"]
for j in range(len(name)):
    bpar = []
    qpar = []
    par = []
    for i in range(len(query)):
        qpar.append(query[i][j])
    for k in range(len(design_space)):
        par.append(design_space[k][j])
    for l in range(len(block)):
        bpar.append(block[l][j])
    plt.title(name[j])
    plt.hist(par,bins=15,density=True,alpha=0.5,label="Random")
    plt.hist(qpar,bins=15,density=True,alpha=0.5,label="Query")
    plt.legend()
    plt.savefig(f"{name[j]}.jpg")
    plt.show()

In [ ]:
# UMAP visualization of query data distribution
import umap
import matplotlib.pyplot as plt
%matplotlib inline
trans = umap.UMAP(random_state=42)
space = trans.fit_transform(design_space)
blk = trans.transform(block)
q = trans.transform(query)
plt.scatter(space[:,0],space[:,1],s=3,c="r",label="random")
plt.scatter(blk[:,0],blk[:,1],s=3,label="block")
plt.scatter(q[:,0],q[:,1],s=50,c="y",label="query")
plt.legend()
plt.savefig("visualize.jpg")
plt.show()